In [11]:
# !pip install openwakeword
# !pip install speechbrain
# !pip install datasets
# !pip install scipy matplotlib

In [12]:
import os
import collections
import numpy as np
from numpy.lib.format import open_memmap
from pathlib import Path

from onnxruntime.transformers.shape_infer_helper import file_path
from tqdm import tqdm
import openwakeword
# import openwakeword.data
import openwakeword.utils
import openwakeword.metrics


import scipy
import datasets
import matplotlib.pyplot as plt
import torch
from torch import nn
import IPython.display as ipd

import datasets
import os
import numpy as np
import scipy.io.wavfile
from tqdm import tqdm

import urllib.request


import warnings

warnings.filterwarnings("ignore")

In [13]:


ds = datasets.load_dataset(
    "speech_commands",
    "v0.02",
    split="test",
    streaming=True
)
ds_iter = iter(ds)

os.makedirs("./data/speech_commands_test", exist_ok=True)
limit = 4890

for i in tqdm(range(limit)):
    output_file = f"./data/speech_commands_test/{i:05d}.wav"

    if os.path.exists(output_file):
        continue

    example = next(ds_iter)
    wav_data = (example["audio"]["array"] * 32767).astype(np.int16) # Convert to 16-bit PCM Format
    scipy.io.wavfile.write(output_file, 16000, wav_data)

100%|██████████| 4890/4890 [00:00<00:00, 16187.49it/s]


# ⇲ Compute Audio Embeddings

Create audio pre-processing object to get openwakeword audio embeddings

In [14]:
model_dir = "./resources/models"
os.makedirs(model_dir, exist_ok=True)

List Model and URL corresponding

In [15]:
models = {
    "embedding_model.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx",
    "embedding_model.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite",
    "melspectrogram.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx",
    "melspectrogram.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite"
}

In [16]:
for filename, url in models.items():
    file_path = os.path.join(model_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"Found {filename}.")


Found embedding_model.onnx.
Found embedding_model.tflite.
Found melspectrogram.onnx.
Found melspectrogram.tflite.


In [17]:
F = openwakeword.utils.AudioFeatures(
    melspec_model_path=f"{model_dir}/melspectrogram.onnx",
    embedding_model_path=f"{model_dir}/embedding_model.onnx",
    inference_framework="onnx",
)

## ⌀ Negative Clips

Get negative example paths, filtering out clips that are too long or too short

 pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cpu

In [18]:
import openwakeword.data